In [ ]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
from PIL import Image

from core.integrate import FeatureFusionScalableTSDFVolume
from core.dataset import ScanNet
from core.labeler import CLIPTextQuerier, KMeansLabeler

In [ ]:
o3d._build_config["ENABLE_HEADLESS_RENDERING"]

# Save fig

In [ ]:
dataset = ScanNet('/home/quanta/Datasets/ScanNet/')
nyu40_color = dataset.nyu40id_to_color
nyu40_class = [dataset.nyu40_id_to_class[i] for i in range(41)]

In [ ]:
scene = "scannet_scene0000_00"
save_dir = "/home/quanta/Experiments/feature-instance-fusion/" + scene + "/"

In [ ]:
json_pth = os.path.abspath("../../config/views/" + scene + ".json")
with open(json_pth, "r") as f:
    views = json.load(f)['views']

In [ ]:
tsdf_device = "cuda:1"
tsdf_volume = FeatureFusionScalableTSDFVolume(
    voxel_size=0.015,
    sdf_trunc=0.075,
    margin=0.08,
    device="cuda:1",
)
tsdf_volume.load(save_dir + "tsdf/tsdf_volume_unpruned.pt")
verts = np.load(save_dir + "tsdf/verts.npy")
faces = np.load(save_dir + "tsdf/faces.npy")
tsdf_volume.load_feats(save_dir + "tsdf_feature_lseg/feats.pt")

In [ ]:
verts_feats = tsdf_volume.extract_feat_on_grid(verts=verts, device='cpu')[0]

In [ ]:
clip_querier = CLIPTextQuerier(device='cuda:1')
clip_querier.load_model()

In [ ]:
labels = clip_querier.multi_text_query(
    texts=nyu40_class,
    img_feats=torch.from_numpy(verts_feats),
)

In [ ]:
mesh = o3d.geometry.TriangleMesh(
    vertices=o3d.utility.Vector3dVector(verts),
    triangles=o3d.utility.Vector3iVector(faces),
)
color = nyu40_color[labels] / 255
mesh.vertex_colors = o3d.utility.Vector3dVector(color)

In [ ]:
save_dir = "../01_LSeg/02_nyu40_multi_class_query/"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
for i, view in enumerate(views):
    vis = o3d.visualization.Visualizer()
    vis.create_window()
    vis.add_geometry(mesh)
    vis.set_view_status(json.dumps(view["view"]))
    buffer = vis.capture_screen_float_buffer(do_render=True)
    image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    # display(image)
    image.save(save_dir + "{:02d}_".format(i) + view["tag"] + ".png")

: 

# Compare to ground truth vertices

In [ ]:
dataset = ScanNet("/home/quanta/Datasets/ScanNet/")
scan_id = "scene0000_00"
id = dataset.scan_id_list.index(scan_id)
single_instance = dataset[id]

In [ ]:
verts = single_instance["vertices"]
faces = single_instance["faces"]
gt_labels = single_instance["ground_truth_labels"]
verts_feats = tsdf_volume.extract_feat_on_grid(verts=verts, device='cpu')[0]
labels = clip_querier.multi_text_query(
    texts=nyu40_class,
    img_feats=torch.from_numpy(verts_feats),
).numpy()

In [ ]:
one_hot_gt = np.zeros((gt_labels.size, 41), dtype=np.int32)
one_hot_gt[np.arange(gt_labels.size), gt_labels] = 1

one_hot_pred = np.zeros((labels.size, 41), dtype=np.int32)
one_hot_pred[np.arange(labels.size), labels] = 1

tp = ((one_hot_gt == one_hot_pred) * one_hot_pred).sum(axis=0)
fp = ((one_hot_gt != one_hot_pred) * one_hot_pred).sum(axis=0)
fn = ((one_hot_gt != one_hot_pred) * (1 - one_hot_pred)).sum(axis=0)
miou = tp / (tp + fp + fn + 1e-16)
mIoU = {}
for cls in dataset.nyu40_id_to_class.keys():
    mIoU[dataset.nyu40_id_to_class[cls]] = miou[cls] * 100
mIoU

# Compared to ground truth labels in finer vertices

In [ ]:
save_dir = "/storage/quanta/Experiments/feature-instance-fusion/" + scene + "/"

verts = np.load(save_dir + "tsdf/verts.npy")
faces = np.load(save_dir + "tsdf/faces.npy")
verts_feats = tsdf_volume.extract_feat_on_grid(verts=verts, device='cpu')[0]
labels = clip_querier.multi_text_query(
    texts=nyu40_class,
    img_feats=torch.from_numpy(verts_feats),
).numpy()

In [ ]:
tsdf_volume.load_feats(save_dir + 'tsdf_feature_gt_semantic/feats.pt')

In [ ]:
gt_score = tsdf_volume.extract_feat_on_grid(verts=verts)[0]

In [ ]:
gt_labels = np.argmax(gt_score, axis=1)

In [ ]:
one_hot_gt = np.zeros((gt_labels.size, 41), dtype=np.int32)
one_hot_gt[np.arange(gt_labels.size), gt_labels] = 1

one_hot_pred = np.zeros((labels.size, 41), dtype=np.int32)
one_hot_pred[np.arange(labels.size), labels] = 1

tp = ((one_hot_gt == one_hot_pred) * one_hot_pred).sum(axis=0)
fp = ((one_hot_gt != one_hot_pred) * one_hot_pred).sum(axis=0)
fn = ((one_hot_gt != one_hot_pred) * (1 - one_hot_pred)).sum(axis=0)
miou = tp / (tp + fp + fn + 1e-16)
mIoU = {}
for cls in dataset.nyu40_id_to_class.keys():
    mIoU[dataset.nyu40_id_to_class[cls]] = miou[cls] * 100
mIoU